In [ ]:
with open(file_name) as f:
    f.readline() # ignore the first line
    f.readline() # ignore the second line
    lines = f.readlines()

In [39]:
import os
import numpy as np
verbose = True
total_rank = 96
num_iter = 5
datasets = ['Cora', 'Citeseer', 'Pubmed', 'Reddit']
my_dict = {datasets[0] + '_L1_XW':[[] for i in range(3*num_iter)], # tot_time_iter1, ..., tot_time_iter5, comm_iter1-5, ..., comp_iter1-5
           datasets[0] + '_L1_AXW':[[] for i in range(3*num_iter)],
           datasets[0] + '_L2_XW':[[] for i in range(3*num_iter)],
           datasets[0] + '_L2_AXW':[[] for i in range(3*num_iter)],
           datasets[1] + '_L1_XW':[[] for i in range(3*num_iter)],
           datasets[1] + '_L1_AXW':[[] for i in range(3*num_iter)],
           datasets[1] + '_L2_XW':[[] for i in range(3*num_iter)],
           datasets[1] + '_L2_AXW':[[] for i in range(3*num_iter)],
           datasets[2] + '_L1_XW':[[] for i in range(3*num_iter)],
           datasets[2] + '_L1_AXW':[[] for i in range(3*num_iter)],
           datasets[2] + '_L2_XW':[[] for i in range(3*num_iter)],
           datasets[2] + '_L2_AXW':[[] for i in range(3*num_iter)],
           datasets[3] + '_L1_XW':[[] for i in range(3*num_iter)],
           datasets[3] + '_L1_AXW':[[] for i in range(3*num_iter)],
           datasets[3] + '_L2_XW':[[] for i in range(3*num_iter)],
           datasets[3] + '_L2_AXW':[[] for i in range(3*num_iter)]
            }
# assign directory
directory = './GCN_results/'
# iterate over files in that directory
for filenm in os.listdir(directory):
    file_name = os.path.join(directory, filenm)
    splitted_file_name = file_name.split("_")
#     print(splitted_file_name)
    dataset = splitted_file_name[1]
    layer = splitted_file_name[2]
    iteration = int(splitted_file_name[3])
    tot_rank = int(splitted_file_name[4].split(".")[0])
#     print(tot_rank)
    if iteration != 1: # ignore the first iteration
        with open(file_name) as f:
            f.readline() # ignore the first line
            f.readline() # ignore the second line
            lines = f.readlines()
            for i in range(tot_rank): # xW
                splitted_lines = lines[i].split(" ")
                rank = int(splitted_lines[0])
                tot_time = float(splitted_lines[2])
                comm_time = float(splitted_lines[3])
                comp_time = float(splitted_lines[4].strip())
                key = dataset + '_' + layer + '_' + 'XW' 
                my_dict[key][(iteration-1)].append(tot_time)
                my_dict[key][(iteration-1) + num_iter].append(comm_time)
                my_dict[key][(iteration-1) + num_iter*2].append(comp_time)
                if verbose == True:
                    print(rank) # rank
                    print(tot_time) # tot_time
                    print(comm_time) # comm
                    print(comp_time) # comp
                
            for i in range(tot_rank): # A(xW)
                j = i + tot_rank
                splitted_lines = lines[j].split(" ")
                rank = int(splitted_lines[0])
                tot_time = float(splitted_lines[2])
                comm_time = float(splitted_lines[3])
                comp_time = float(splitted_lines[4].strip())
                key = dataset + '_' + layer + '_' + 'AXW' 
                my_dict[key][(iteration-1)].append(tot_time)
                my_dict[key][(iteration-1) + num_iter].append(comm_time)
                my_dict[key][(iteration-1) + num_iter*2].append(comp_time)
                if verbose == True:
                    print(rank)
                    print(tot_time) # tot_time
                    print(comm_time) # comm
                    print(comp_time) # comp
            # Automatically ignore the last line.
# now my_dict is ready!
for key in my_dict:
    result_list = my_dict[key]
    for j in range(3*num_iter):
        max_result = max(result_list[j])
        min_result = min(result_list[j])
        if key[-4:-1] == 'AXW': # otherwise if its XW we will not accelerate it
            time_stamp = np.array(result_list[j]) - min_result
            gcn_fpga_time = # depends on the rank b/c different ranks have different packet length
            skew_time = comp_delay(time_stamp, , total_rank, 'gcn', gcn_fpga_time)
            gcn_cpu_time = min_result
        print('max: ', max_result)
        print('min: ', min_result)
        

['./dummy/GCN', 'Citeseer', 'L1', '1', '96.txt']
96
0
0.00168146
0.00165196
2.9496e-05
1
0.00167064
0.00164323
2.7409e-05
2
0.00180963
0.0017811
2.8528e-05
3
0.00168182
0.00165393
2.7884e-05
4
0.00160491
0.00157649
2.8426e-05
5
0.00160205
0.00157458
2.7467e-05
6
0.00164132
0.00161029
3.1032e-05
7
0.00168998
0.00166418
2.5809e-05
8
0.00167135
0.00164405
2.7295e-05
9
0.00181127
0.00178445
2.6822e-05
10
0.00175023
0.0016891
6.1131e-05
11
0.00179358
0.00176602
2.7556e-05
12
0.0016718
0.00164502
2.6783e-05
13
0.00174971
0.00172377
2.5934e-05
14
0.00168367
0.00165472
2.8954e-05
15
0.00163101
0.00160386
2.7151e-05
16
0.00181297
0.00178674
2.6232e-05
17
0.00170525
0.00167671
2.8536e-05
18
0.00180236
0.00177366
2.8695e-05
19
0.00172416
0.00169764
2.6515e-05
20
0.00175969
0.00173145
2.8231e-05
21
0.00184491
0.00181763
2.7283e-05
22
0.00178529
0.00175949
2.5796e-05
23
0.00181051
0.00178341
2.7101e-05
24
0.00179834
0.00177064
2.77e-05
25
0.00173721
0.00170853
2.8673e-05
26
0.00170558
0.00167733
2.

In [ ]:
#another for loop for different num_procs


In [8]:
mydict = {'apple':[[], []], 'orange':[[4,5,6],[5,6,7]]}
key = 'app' +'le'
mydict[key][0].append(3.14)
mydict['apple']

[[3.14], []]

In [10]:
a = [[] for i in range(5)]
print(a)

[[], [], [], [], []]


In [1]:
mydict = {'apple':[[], []], 'orange':[[4,5,6],[5,6,7]]}
for key in mydict:
    print(key)

apple
orange


In [6]:
import numpy as np
a= [2, 34, 55, 6]
np.array(a)-min(a)

array([ 0, 32, 53,  4])

In [9]:
a='hjghg_AXW'
a[-3:-1]

'AX'